In [1]:
import os
import matplotlib.pyplot as plt
from PIL import Image, ImageEnhance
import easyocr
import pandas as pd
import cv2
import numpy as np

In [2]:
reader = easyocr.Reader(["en"],gpu=False)

folder_path = 'test_images'

csv_path = './updated_test_values.csv'
labels_df = pd.read_csv(csv_path)

Using CPU. Note: This module is much faster with a GPU.


In [3]:
ground_truths = dict(zip(labels_df['image_name'], labels_df['label']))

In [7]:
# Klasördeki sadece .png uzantılı dosyaları listeleme
files = [filename for filename in os.listdir(folder_path) if filename.endswith('.png') and os.path.isfile(os.path.join(folder_path, filename))]

correct = 0
total = 0

# Preprocessing ve resize fonksiyonu
def preprocess_and_resize_image(image_path, target_size):
    """
    Preprocess an image and resize without distortion.
    Args:
        image_path (str): Path to the image file.
        target_size (tuple): Desired output image size (width, height).
    Returns:
        np.array: Preprocessed and resized image.
    """
    # Resmi OpenCV ile oku
    img = cv2.imread(image_path)

    # Orijinal boyutları al
    original_height, original_width = img.shape[:2]

    # Siyah beyaz yapma
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Kontrast artırma
    enhancer = ImageEnhance.Contrast(Image.fromarray(gray_img))
    enhanced_img = np.array(enhancer.enhance(6))  # 6x kontrast artırma

    # Threshold uygulama
    _, threshold_img = cv2.threshold(enhanced_img, 175, 255, cv2.THRESH_BINARY)

    # Resize işlemi
    target_width, target_height = target_size
    scale = min(target_width / original_width, target_height / original_height)  # Ölçek faktörü

    resized_width = int(original_width * scale)
    resized_height = int(original_height * scale)

    # Görüntüyü boyutlandırma
    resized_img = cv2.resize(threshold_img, (resized_width, resized_height), interpolation=cv2.INTER_AREA)

    # Padding ekleme
    pad_top = (target_height - resized_height) // 2
    pad_bottom = target_height - resized_height - pad_top
    pad_left = (target_width - resized_width) // 2
    pad_right = target_width - resized_width - pad_left

    padded_img = cv2.copyMakeBorder(resized_img, pad_top, pad_bottom, pad_left, pad_right, cv2.BORDER_CONSTANT, value=255)

    return padded_img

# OCR ve değerlendirme işlemi
for file in files:
    image_path = os.path.join(folder_path, file)

    # Görüntüye preprocessing ve resize işlemi uygula
    preprocessed_image = preprocess_and_resize_image(image_path, target_size=(224, 224))  # Örneğin 224x224 hedef boyutu

    # EasyOCR ile metni çıkartın
    result = reader.readtext(preprocessed_image)
    extracted_text = " ".join([text[1] for text in result]) if result else ""  # Metinleri birleştir, eğer sonuç yoksa boş bir metin

    # Gerçek metni almak için ground_truths sözlüğünden doğru etiketi alın
    ground_truth = ground_truths.get(file, "")

    # Eğer OCR tarafından çıkarılan metin boşsa, bunu yanlış kabul edelim
    if not extracted_text:
        is_correct = False  # OCR sonucu boşsa yanlış kabul edelim
    else:
        is_correct = (extracted_text.strip().lower() == ground_truth.strip().lower())  # Case-insensitive karşılaştırma

    # Toplam ve doğru sayıları güncelle
    total += 1
    if is_correct:
        correct += 1

    # Değerlendirme (evaluation) metriklerini hesaplayın
    print(f"Evaluation for {file}:")
    print(f"Extracted Text: {extracted_text}")
    print(f"Ground Truth: {ground_truth}")
    print(f"Correct: {is_correct}")
    print("====================================")

# Accuracy hesaplama
accuracy = correct / total if total > 0 else 0
print(f"Overall Accuracy: {accuracy:.4f}")


Evaluation for a01-000u-01-02.png:
Extracted Text: Lone
Ground Truth: more
Correct: False
Evaluation for a01-003-03-00.png:
Extracted Text: 75
Ground Truth: Foot's
Correct: False
Evaluation for a01-003-06-01.png:
Extracted Text: 7l
Ground Truth: put
Correct: False
Evaluation for a01-003x-04-06.png:
Extracted Text: mk
Ground Truth: into
Correct: False
Evaluation for a01-003x-08-04.png:
Extracted Text: Lp
Ground Truth: up
Correct: False
Evaluation for a01-007-04-04.png:
Extracted Text: Ol
Ground Truth: an
Correct: False
Evaluation for a01-007-08-02.png:
Extracted Text: 
Ground Truth: ,
Correct: False
Evaluation for a01-007u-05-01.png:
Extracted Text: lur_L~Qs)
Ground Truth: members
Correct: False
Evaluation for a01-011-05-04.png:
Extracted Text: buk
Ground Truth: but
Correct: False
Evaluation for a01-011-06-02.png:
Extracted Text: Hachod
Ground Truth: Macleod
Correct: False
Evaluation for a01-011u-09-00.png:
Extracted Text: La
Ground Truth: is
Correct: False
Evaluation for a01-014-08-03.